### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('./Datasets/movies_clean.csv')
reviews = pd.read_csv('./Datasets/reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [2]:
reviews.head()

,user_id,movie_id,rating,timestamp,date_time
0,1,114508,8,1381006850,2013-10-06 02:30:50
1,2,358273,9,1579057827,2020-01-15 08:40:27
2,2,10039344,5,1578603053,2020-01-10 02:20:53
3,2,6751668,9,1578955697,2020-01-14 04:18:17
4,2,7131622,8,1579559244,2020-01-21 03:57:24


In [4]:
def create_ranked_df(movies, reviews):
    avg_rating = reviews.groupby('movie_id')['rating'].mean()
    num_rating = reviews.groupby('movie_id')['rating'].count()
    latest_rating = reviews.groupby('movie_id').max()['date_time']

    ratings_detail = pd.DataFrame({'avg_rating': avg_rating, 'num_rating': num_rating, 'last_rating': latest_rating})
    
    movie_recs = movies.set_index('movie_id').join(ratings_detail)
    
    ranked_movies = movie_recs.sort_values(['avg_rating', 'num_rating', 'last_rating'], ascending=False)
    
    ranked_movies = ranked_movies[ranked_movies['num_rating'] > 4]
    
    return ranked_movies

def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    top_movies = list(ranked_movies['movie'][:n_top])
    
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [9]:
# Put your solutions for each of the cases here
ranked_movies = create_ranked_df(movies, reviews)
# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20) # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5) # Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100) # Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35) # Your solution list here

In [10]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [11]:
ranked_movies.head()

,movie,genre,date,1800's,1900's,2000's,Western,Reality-TV,Fantasy,Comedy,...,Romance,Adult,News,Drama,Music,History,Short,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy|Horror,2015,0,0,1,0,0,1,1,...,0,0,0,1,0,0,0,10.0,48,2016-08-14 22:46:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,10.0,28,2016-01-08 06:14:43
6662050,Five Minutes (2017),Short|Comedy,2017,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,10.0,22,2019-04-21 03:59:19
2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,10.0,10,2015-05-11 04:26:01
5804314,Let There Be Light (2017),Drama,2017,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,7,2019-12-25 21:57:47


In [15]:
def popular_recs_filtered(user_id, n_top, ranked_movies, genres, years):
    
    ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]
    
    ranked_movies = ranked_movies[ranked_movies[genres].sum(axis=1) > 0]
    
    top_movies = list(ranked_movies['movie'][:n_top])
    
    return top_movies

In [16]:
top_movies = popular_recs_filtered(1, 20, ranked_movies, ['History'], ['2015', '2016', '2017', '2018'])

In [17]:
top_movies

["Hillary's America: The Secret History of the Democratic Party (2016)",
 'I Believe in Miracles (2015)',
 'O.J.: Made in America (2016)',
 'Ayla: The Daughter of War (2017)',
 'They Shall Not Grow Old (2018)',
 'Hacksaw Ridge (2016)',
 'Namhansanseong (2017)',
 'The Farthest (2017)',
 'Kono sekai no katasumi ni (2016)',
 'Sado (2015)',
 'Silicon Cowboys (2016)',
 '13th (2016)',
 'Ethel &amp; Ernest (2016)',
 'Paul, Apostle of Christ (2018)',
 'Kincsem (2017)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Nise - O CoraÃ§Ã£o da Loucura (2015)',
 'Under sandet (2015)',
 'Only the Dead (2015)']